In [1]:
import torch

from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor

d:\programfiles\Anaconda\anaconda\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Download training data from open datasets.
traning_data = datasets.FashionMNIST(
    root='data', 
    train=True, 
    download=True, 
    transform=ToTensor()
    )

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root='data', 
    train=False, 
    download=True, 
    transform=ToTensor()
)


In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(traning_data, batch_size=batch_size )
test_dataloader = DataLoader(test_data, batch_size=batch_size )

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# create model
device = (
    "cuda" 
    if torch.cuda.is_available()
    else "cpu"
)
print(f'Using {device} device')



Using cuda device


In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device=device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f'loss: {loss:>7f} [{current:>5d} / {size:>5d}]')
            

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss:{test_loss:>8f}\n')

In [10]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t + 1} \n -------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
 -------------------------------
loss: 2.302062 [   64 / 60000]
loss: 2.282287 [ 6464 / 60000]
loss: 2.266757 [12864 / 60000]
loss: 2.261897 [19264 / 60000]
loss: 2.225316 [25664 / 60000]
loss: 2.197914 [32064 / 60000]
loss: 2.213953 [38464 / 60000]
loss: 2.176410 [44864 / 60000]
loss: 2.174870 [51264 / 60000]
loss: 2.125652 [57664 / 60000]
Test Error: 
 Accuracy: 37.1%, Avg loss:2.129580

Epoch 2 
 -------------------------------
loss: 2.151627 [   64 / 60000]
loss: 2.129806 [ 6464 / 60000]
loss: 2.074414 [12864 / 60000]
loss: 2.093127 [19264 / 60000]
loss: 2.014099 [25664 / 60000]
loss: 1.961733 [32064 / 60000]
loss: 1.999536 [38464 / 60000]
loss: 1.913490 [44864 / 60000]
loss: 1.922060 [51264 / 60000]
loss: 1.835779 [57664 / 60000]
Test Error: 
 Accuracy: 48.9%, Avg loss:1.838841

Epoch 3 
 -------------------------------
loss: 1.885498 [   64 / 60000]
loss: 1.838495 [ 6464 / 60000]
loss: 1.727560 [12864 / 60000]
loss: 1.776111 [19264 / 60000]
loss: 1.639324 [25664 / 60000]

In [11]:
# saving models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
# Loading Model
model = NeuralNetwork().to(device)  # 要先创建一个模型，才能load进数据
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()   # 把模型改为推理状态
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
